In [67]:
# Libraries
import time
import pandas as pd 
from random import randint
from IPython.display import clear_output

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
# Driver path
path = '/Users/kritixblaze/Developer/chromedriver_mac64/chromedriver'
driver = webdriver.Chrome(path)

/var/folders/4y/pcdt74h51_x2zz4w3hlpvzxh0000gn/T/ipykernel_29356/660491345.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [ ]:
# Login credentials

with open('config.txt', 'r') as file:
    email, password = file.readlines()

In [3]:
# Maximize Window
driver.minimize_window()  
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

In [4]:
# Enter to the site
driver.get('https://www.linkedin.com/login')
time.sleep(2)

# User Credentials
driver.find_element('xpath', '//*[@id="username"]').send_keys(email[:-1])
driver.find_element('xpath', '//*[@id="password"]').send_keys(password)
time.sleep(1)

In [5]:
# Login button
driver.find_element('xpath', '//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(30)

In [6]:
# Jobs page
# driver.find_element('xpath', '//*[@id="ember19"]').click()
driver.find_element('xpath', '//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
time.sleep(3)

In [97]:
# Keyword
keywords = input("Enter keywords: ").lower()

Enter keywords:  qa tester


In [98]:
keywords = keywords.strip().replace(" ", "%20")

In [99]:
link = f"https://www.linkedin.com/jobs/search/?keywords={keywords}&location=India&f_E=2"

In [100]:
link

'https://www.linkedin.com/jobs/search/?keywords=qa%20tester&location=India&f_E=2'

In [101]:
driver.get(link)
time.sleep(1)

In [102]:
links = []
print("Collecting links...")

try:
    for page in range(2, 15):
        time.sleep(randint(1, 4))
        jobs_block = driver.find_element('xpath', '//*[@id="main"]/div/section[1]/div/ul')
        jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
    
        for job in jobs_list:
            all_links = job.find_elements('tag name', 'a')
            for a in all_links:
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links: 
                    links.append(a.get_attribute('href'))
                else:
                    pass
            # scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", job)
        
        clear_output(wait=True)
        print(f'Collecting the links in the page: {page-1}')
        # go to next page:
        driver.find_element('xpath', f"//button[@aria-label='Page {page}']").click()        

except Exception as e:
    print(str(e))

In [103]:
df = pd.DataFrame(links)
df.head()

,0
0,https://www.linkedin.com/jobs/view/3423104283/...
1,https://www.linkedin.com/jobs/view/3422999705/...
2,https://www.linkedin.com/jobs/view/3348013200/...
3,https://www.linkedin.com/jobs/view/3442837302/...
4,https://www.linkedin.com/jobs/view/3337878939/...


In [104]:
df.shape

(321, 1)

In [105]:
filename = "links/" + keywords.replace("%20", "_") + "_offers.csv"

In [106]:
filename

'links/qa_tester_offers.csv'

In [107]:
# Storing the data to csv file
df.to_csv(filename, index=False)